In [1]:
from torchvision import datasets
import torchvision.transforms as transforms
import torch
from torch import autograd
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torch.distributions as td
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

torch.cuda.device_count()

0

In [2]:
noise = torch.rand(28,28).to(device)

batch_size = 64


transform = transforms.ToTensor()


train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)

tl = torch.utils.data.DataLoader(train_data, batch_size=2)

In [3]:
#dataiter = iter(train_loader)
#images, labels = dataiter.next()
#images = images.numpy()

In [4]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 64, 3, stride = 1)
        self.conv2 = nn.Conv2d(64, 64, 3, stride = 2)
        self.conv3 = nn.Conv2d(64, 128, 3, stride = 1)            
        self.conv4 = nn.Conv2d(128, 128, 3, stride = 2)
        #self.conv5 = nn.Conv2d(128, 256, 3, stride = 1)  #MNIST SMALL FOR THIS NTWORK
        #self.conv6 = nn.Conv2d(256, 256, 3, stride = 2)  #END UP WITH IMAGE SIZE SMALLER THAN THE KERNEL SIZE 
        self.relu = nn.LeakyReLU(0.2, inplace=True)          
        self.layer1 = torch.nn.Linear(2048, 1024)
        self.layer2 = torch.nn.ReLU()
        self.layer3 = torch.nn.Linear(1024, 1)
        #self.layer4 = torch.nn.Sigmoid() 

    def forward(self, input):
        
        out = self.relu(self.conv1(input))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out = self.relu(self.conv4(out))
        #out = self.relu(self.conv5(out))
        #out = self.relu(self.conv6(out))
        out = out.view(out.shape[0], -1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        #out = self.layer4(out)
        
        return out
      

In [5]:

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        #self.conv3 = nn.Conv2d(32, 32, 3)
        #self.conv4 = nn.Conv2d(32, 32, 3)
        #self.deconv1 = nn.ConvTranspose2d(32,32,3)
        #self.deconv2 = nn.ConvTranspose2d(64,32,3)
        self.deconv3 = nn.ConvTranspose2d(64,32,3)
        self.deconv4 = nn.ConvTranspose2d(64,1,3)
            
            
    def forward(self, x):
        
      x  = F.relu(self.conv1(x))
      x1 = x
      x  = F.relu(self.conv2(x))
      x2 = x
      #x  = F.relu(self.conv3(x))
      #x3 = x
      #x  = F.relu(self.conv4(x))
      #x  = F.relu(self.deconv1(x))
      #x  = F.relu(self.deconv2(torch.cat([x3, x], dim=1)))
      x  = F.relu(self.deconv3(torch.cat([x2, x], dim=1)))
      x  = F.relu(self.deconv4(torch.cat([x1, x], dim=1)))
      return x


In [6]:
lambda_pen=10

In [9]:
# instantiate discriminator and generator
D = Discriminator()
G = Generator()
D.to(device)
G.to(device)

Generator(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (deconv3): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(1, 1))
  (deconv4): ConvTranspose2d(64, 1, kernel_size=(3, 3), stride=(1, 1))
)

In [10]:
import torch.optim as optim

# Optimizers
lr = 0.002

# wgan optimizers Create optimizers for the discriminator and generator
#D_optimizer = optim.RMSprop(D.parameters(), lr)
#G_optimizer = optim.RMSprop(G.parameters(), lr)


###### wgan-gp optimizer ########
D_optimizer = optim.Adam(D.parameters(), lr)
G_optimizer = optim.Adam(G.parameters(), lr)

In [11]:

def gauss(args) :
  return args + noise

In [ ]:



# training hyperparams
num_epochs = 4000
num_critic = 10

# keep track of loss and generated, "fake" samples
samples = []
losses = []

print_every = 400

# Get some fixed data for sampling. These are images that are held
# constant throughout training, and allow us to inspect the model's performance : LDCT_test

x, _ = next(iter(tl))
LDCT_test = x.to(device) + noise

# uniform = torch.distributions.uniform.Uniform(-1, 1, validate_args=None)




# train the network
D.train()
G.train()
for epoch in range(num_epochs):
  
    ii = 1
    
    for batch_i, (ND_CT, _) in enumerate(train_loader):
   
        # train D
        
        
        NDCT = ND_CT.to(device)
        D_optimizer.zero_grad()
        
        # 1. Train with NDCT

        # Compute the discriminator losses on NDCT images 
        
        D_NDCT = D(NDCT)
        
        
        # 2. Train with LDCT images
        
        # Generate LDCT images
        LDCT = gauss(NDCT)
        G_LDCT = G(LDCT)
        D_LDCT = D(G_LDCT)
        
        ############# wgan gp ###################
        
        epsilon = torch.distributions.uniform.Uniform(0, 1, validate_args=None).sample()
            
        interpolated = epsilon * NDCT + (1 - epsilon) * G_LDCT 
        
        
        gradients = autograd.grad(outputs=D(interpolated), inputs=interpolated,
                               grad_outputs=torch.ones(D(interpolated).size()).to(device),
                               create_graph=True, retain_graph=True)[0]
             

        grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * lambda_pen
        
        
        D_loss = -D_LDCT.mean() + D_NDCT.mean() + grad_penalty
            
                        
        # wgan loss :  D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        D_loss.backward(retain_graph=True)
        D_optimizer.step()
                                         
        # train G
        
        
        
        # 1. Train with denoised images and flipped labels
        
        if ii == num_critic :
          
          G_optimizer.zero_grad()
          
          NDCT = ND_CT.to(device)
          LDCT = gauss(NDCT)
          
          # Generate denoised images
          G_LDCT = G(LDCT)
          D_LDCT = D(G_LDCT)
          G_loss = - D_LDCT.mean()
        
          # perform backprop
          G_loss.backward(retain_graph=True)
          G_optimizer.step()
          ii = 0
          
        ii = ii + 1  

        # Print some loss stats
        if batch_i % print_every == 10:
            # print discriminator and generator loss
            print('Epoch [{:5d}/{:5d}] | D_loss: {:6.4f} | G_loss: {:6.4f}'.format(
                    epoch+1, num_epochs, D_loss.item(), G_loss.item()))

    
    ## after each epoch ##
    # append discriminator loss and generator loss
    losses.append((D_loss.item(), G_loss.item()))
    
    # generate and save sample, denoised images
    G.eval() # eval mode for generating samples
    denoised = G(LDCT_test).cpu()
    plt.imshow(denoised.detach()[0].numpy()[0], cmap='gray')
    samples.append(denoised)
    G.train() # back to train mode

In [ ]:
 plt.imshow(LDCT_test[0].cpu().numpy()[0], cmap='gray')
 

In [0]:
plt.imshow(x[0].numpy()[0], cmap='gray')

In [0]:
samples = torch.zeros(0, 0)

In [0]:
x, _ = next(iter(tl))
LDCT_test = x.to(device) + noise


In [0]:
 plt.imshow(denoised.detach()[0].numpy()[0], cmap='gray')

In [0]:
denoised.shape


In [0]:
batch_i